# Assignment 10

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

    在计算likelihood所有特征的联合概率分布时假设每个特征的发生是独立无关的，可以分解为所有特征发生概率的乘积

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

    最大化后验概率即最大化likelihood与先验概率的乘积，当我们假设每种先验假设发生的概率相同时，可以简化为最大化likelihood

#### 3. What is support vector in SVM?

    在正负例两侧的分割边界上的数据点

#### 4. What is the intuition behind SVM ?

    找到一个分割平面，使得该平面距离两侧最近的正负例间隔最大

#### 5. Shortly describ what 'random' means in random forest ?

    随机取数据集中的部分数据进行学习，每棵树的数据都是有放回的取；每棵树都是随机选择部分特征进行学习

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?

    我的理解是先给定一个计算预估值与真实值的L2损失函数与L2正则项，其中总预估值可以表示为前面所有树的总和加上当前树的学习值，前面所有树的正则项合并到常数项，此时正则项为当前树的正则项，可以将前面所有树的总和当做x，将当前树的学习值当做Δx进行二阶泰勒展开，二阶的平方项可与L2正则项合并（此时正则项还剩一个系数乘叶子节点数量），此时是遍历所有叶子节点（每个节点中的样本的学习值相同），再对每个节点的样本遍历。将每个节点的样本遍历的一阶项总和当做G，二阶项总和当做H，对当前学习值w求导取极值可解出w=-G/(H+λ)，代入原式中可当做一个树的标准（越小越好）；将分之前的树的标准与分之后的树的标准做差当做判断的增益，增益越小越好

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

In [19]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
import ast

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
data = pd.read_csv('sqlResult_1558435.csv', encoding='utf-8')
data.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走）\r\n@深圳交警微博称：昨日清晨交...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [3]:
data['publisher'] = np.where(data['source'] == '新华社', 1, 0)

##### 查看y的分布，大比例为“新华社”

In [4]:
data['publisher'].value_counts()

1    78661
0    10950
Name: publisher, dtype: int64

##### 查看content是否存在异常值

In [5]:
data['content'].isnull().sum()

2565

In [6]:
data.dropna(subset=['content'], inplace=True)

##### 先导入停用词，并整理为列表格式

In [7]:
stopwords = [word.strip() for word in open('哈工大停用词表.txt',encoding='utf-8') if word.strip() != '']

##### 将‘content’进行分词处理，并当做x数据集

In [8]:
sentences = []
start_time = time.time()

for content in data['content']:
    sentence = [word for word in jieba.cut(content) if word not in stopwords and word not in ['\u3000', '\n','\r\n']]
    sentences.append(sentence)
    
end_time = time.time()
print(f'所有文章分词共用时{end_time - start_time}秒')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3t/rkcyv7gj2zzc07y_vbtsz8lm0000gn/T/jieba.cache
Loading model cost 0.583 seconds.
Prefix dict has been built succesfully.


所有文章分词共用时283.06476521492004秒


##### 将分词结果保存

In [9]:
with open('cutwords.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
       f.write(' '.join(sentence) + '\n')

##### 直接读取分词结果

In [10]:
with open('cutwords.txt', 'r', encoding='utf-8') as f:
    sentences = [line.split() for line in f]

##### 对文本进行TFIDF建模

In [11]:
tfidf_model = TfidfVectorizer(sentences, min_df=50, max_df=0.3)

##### X

In [12]:
X = tfidf_model.fit_transform(data['content'])
print(f'词表维度:{len(tfidf_model.vocabulary_)}')
print(f'数据集X维度:{X.shape}')

词表维度:2489
数据集X维度:(87046, 2489)


##### Y

In [13]:
Y = data['publisher'].values

##### 采用分层抽样划分数据集

In [14]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, stratify=Y)
print(f'训练集X数量为：{train_x.shape[0]}')
print(f'测试集X数量为：{test_x.shape[0]}')

训练集X数量为：69636
测试集X数量为：17410


In [15]:
from sklearn.naive_bayes import MultinomialNB

In [16]:
bayes_model = MultinomialNB()
bayes_model.fit(train_x, train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
print(f'Accuracy with Naive Bayes: {bayes_model.score(test_x, test_y)}')
y_pred_bayes = bayes_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with Naive Bayes: 0.9511774842044802
              precision    recall  f1-score   support

        非新华社       0.73      0.77      0.75      1677
         新华社       0.98      0.97      0.97     15733

    accuracy                           0.95     17410
   macro avg       0.85      0.87      0.86     17410
weighted avg       0.95      0.95      0.95     17410



,非新华社,新华社
非新华社,1296,381
新华社,469,15264


##### 数据不平衡，大比例为新华社，非新华社的分类效果不好。试采用SMOTE上采样

In [20]:
X_smote1, Y_smote1 = SMOTE().fit_sample(train_x, train_y)

In [21]:
bayes_model.fit(X_smote1, Y_smote1)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
print(f'Accuracy with Naive Bayes: {bayes_model.score(test_x, test_y)}')
y_pred_bayes = bayes_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with Naive Bayes: 0.8668006892590465
              precision    recall  f1-score   support

        非新华社       0.42      0.98      0.59      1677
         新华社       1.00      0.85      0.92     15733

    accuracy                           0.87     17410
   macro avg       0.71      0.92      0.75     17410
weighted avg       0.94      0.87      0.89     17410



,非新华社,新华社
非新华社,1643,34
新华社,2285,13448


In [25]:
X_smote2, Y_smote2 = BorderlineSMOTE(kind='borderline-1').fit_sample(train_x, train_y)

In [26]:
bayes_model.fit(X_smote2, Y_smote2)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
print(f'Accuracy with Naive Bayes: {bayes_model.score(test_x, test_y)}')
y_pred_bayes = bayes_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with Naive Bayes: 0.8554279149913843
              precision    recall  f1-score   support

        非新华社       0.40      0.99      0.57      1677
         新华社       1.00      0.84      0.91     15733

    accuracy                           0.86     17410
   macro avg       0.70      0.91      0.74     17410
weighted avg       0.94      0.86      0.88     17410



,非新华社,新华社
非新华社,1652,25
新华社,2492,13241


In [28]:
X_smote3, Y_smote3 = BorderlineSMOTE(kind='borderline-2').fit_sample(train_x, train_y)

In [29]:
bayes_model.fit(X_smote3, Y_smote3)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
print(f'Accuracy with Naive Bayes: {bayes_model.score(test_x, test_y)}')
y_pred_bayes = bayes_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with Naive Bayes: 0.8593337162550259
              precision    recall  f1-score   support

        非新华社       0.40      0.91      0.55      1677
         新华社       0.99      0.85      0.92     15733

    accuracy                           0.86     17410
   macro avg       0.69      0.88      0.74     17410
weighted avg       0.93      0.86      0.88     17410



,非新华社,新华社
非新华社,1521,156
新华社,2293,13440


##### 使用SMOTE总体上Accuracy会降低，但对于“非新华社”的recall会显著提升

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).

### 逻辑回归模型

In [31]:
from sklearn.linear_model import LogisticRegressionCV

In [36]:
C = [1, 10, 100, 1000]
lr_model = LogisticRegressionCV(Cs=C, class_weight='balanced')
lr_model.fit(train_x, train_y)

LogisticRegressionCV(Cs=[1, 10, 100, 1000], class_weight='balanced', cv=None,
                     dual=False, fit_intercept=True, intercept_scaling=1.0,
                     l1_ratios=None, max_iter=100, multi_class='auto',
                     n_jobs=None, penalty='l2', random_state=None, refit=True,
                     scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [38]:
print(f'Accuracy with LogisticRegression: {lr_model.score(test_x, test_y)}')
y_pred_bayes = lr_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with LogisticRegression: 0.9085008615738082
              precision    recall  f1-score   support

        非新华社       0.51      0.94      0.66      1677
         新华社       0.99      0.91      0.95     15733

    accuracy                           0.91     17410
   macro avg       0.75      0.92      0.81     17410
weighted avg       0.95      0.91      0.92     17410



,非新华社,新华社
非新华社,1572,105
新华社,1488,14245


In [174]:
lr_model.fit(X_smote2, Y_smote2)

LogisticRegressionCV(Cs=[1, 10, 100, 1000], class_weight='balanced', cv=None,
                     dual=False, fit_intercept=True, intercept_scaling=1.0,
                     l1_ratios=None, max_iter=100, multi_class='auto',
                     n_jobs=None, penalty='l2', random_state=None, refit=True,
                     scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [175]:
print(f'Accuracy with LogisticRegression: {lr_model.score(test_x, test_y)}')
y_pred_lr = lr_model.predict(test_x)
print(classification_report(test_y, y_pred_lr, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_lr), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with LogisticRegression: 0.9126938541068351
              precision    recall  f1-score   support

        非新华社       0.53      0.92      0.67      1677
         新华社       0.99      0.91      0.95     15733

    accuracy                           0.91     17410
   macro avg       0.76      0.92      0.81     17410
weighted avg       0.95      0.91      0.92     17410



,非新华社,新华社
非新华社,1551,126
新华社,1394,14339


### 支持向量机模型

In [40]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [41]:
svc_param = {'C': [0.1, 1, 10, 100],
         'kernel': ['linear', 'rbf'],
         'class_weight': ['balanced']}

cv = StratifiedKFold(shuffle=True)
clf = SVC()
cv_svc = GridSearchCV(clf, svc_param, cv=cv)
cv_svc.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100], 'class_weight': ['balanced'],
                         'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [42]:
cv_svc.best_params_

{'C': 10, 'class_weight': 'balanced', 'kernel': 'rbf'}

In [45]:
svc_model = SVC(C=10, class_weight='balanced', kernel='rbf')
svc_model.fit(train_x, train_y)

SVC(C=10, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [46]:
print(f'Accuracy with SVM: {svc_model.score(test_x, test_y)}')
y_pred_bayes = svc_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with SVM: 0.9276852383687536
              precision    recall  f1-score   support

        非新华社       0.59      0.85      0.69      1677
         新华社       0.98      0.94      0.96     15733

    accuracy                           0.93     17410
   macro avg       0.78      0.89      0.83     17410
weighted avg       0.94      0.93      0.93     17410



,非新华社,新华社
非新华社,1418,259
新华社,1000,14733


In [176]:
svc_model.fit(X_smote2, Y_smote2)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [177]:
print(f'Accuracy with SVM: {svc_model.score(test_x, test_y)}')
y_pred_svm = svc_model.predict(test_x)
print(classification_report(test_y, y_pred_svm, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_svm), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with SVM: 0.9274554853532453
              precision    recall  f1-score   support

        非新华社       0.58      0.86      0.70      1677
         新华社       0.98      0.93      0.96     15733

    accuracy                           0.93     17410
   macro avg       0.78      0.90      0.83     17410
weighted avg       0.95      0.93      0.93     17410



,非新华社,新华社
非新华社,1447,230
新华社,1033,14700


### 随机森林模型

In [47]:
from sklearn.ensemble import RandomForestClassifier

##### 先对n_estimators调参

In [66]:
rf_param = {'n_estimators': [200, 400, 600],
         'min_samples_split': [18],
         'max_depth' : [14]}

cv = StratifiedKFold(shuffle=True)
clf = RandomForestClassifier()
cv_rf = GridSearchCV(clf, rf_param, cv=cv)
cv_rf.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                          

In [67]:
cv_rf.best_params_

{'max_depth': 14, 'min_samples_split': 18, 'n_estimators': 200}

In [68]:
cv_rf.best_score_

0.913722795653895

##### 固定n_estimators，对min_samples_split与max_depth调参

##### 经过多次调参，最后确定的参数如下：

In [117]:
rf_param = {'n_estimators': [200],
         'min_samples_split': [36, 40, 44],
         'max_depth' : [260, 270, 280]}

cv = StratifiedKFold(shuffle=True)
clf = RandomForestClassifier()
cv_rf = GridSearchCV(clf, rf_param, cv=cv)
cv_rf.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                          

In [118]:
cv_rf.best_params_

{'max_depth': 280, 'min_samples_split': 36, 'n_estimators': 200}

In [119]:
cv_rf.best_score_

0.9594462471027747

##### 对max_features调参

In [123]:
rf_param = {'n_estimators': [200],
         'min_samples_split': [36],
         'max_depth': [280],
         'max_features': ["auto", "sqrt", "log2"]}

cv = StratifiedKFold(shuffle=True)
clf = RandomForestClassifier()
cv_rf = GridSearchCV(clf, rf_param, cv=cv)
cv_rf.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                          

In [124]:
cv_rf.best_params_

{'max_depth': 280,
 'max_features': 'auto',
 'min_samples_split': 36,
 'n_estimators': 200}

In [125]:
cv_rf.best_score_

0.959733458993347

In [128]:
rf_model = RandomForestClassifier(n_estimators=200, min_samples_split=36, max_features='auto', max_depth=280)
rf_model.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=280, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=36,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [129]:
print(f'Accuracy with RandomForest: {rf_model.score(test_x, test_y)}')
y_pred_bayes = rf_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with RandomForest: 0.9591614014933946
              precision    recall  f1-score   support

        非新华社       0.91      0.64      0.75      1677
         新华社       0.96      0.99      0.98     15733

    accuracy                           0.96     17410
   macro avg       0.94      0.82      0.86     17410
weighted avg       0.96      0.96      0.96     17410



,非新华社,新华社
非新华社,1074,603
新华社,108,15625


In [178]:
rf_model.fit(X_smote2, Y_smote2)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=280, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=36,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [179]:
print(f'Accuracy with RandomForest: {rf_model.score(test_x, test_y)}')
y_pred_rf = rf_model.predict(test_x)
print(classification_report(test_y, y_pred_rf, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_rf), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with RandomForest: 0.9004595060310167
              precision    recall  f1-score   support

        非新华社       0.49      0.92      0.64      1677
         新华社       0.99      0.90      0.94     15733

    accuracy                           0.90     17410
   macro avg       0.74      0.91      0.79     17410
weighted avg       0.94      0.90      0.91     17410



,非新华社,新华社
非新华社,1549,128
新华社,1605,14128


### xgboost模型

In [130]:
from xgboost import XGBClassifier

##### 先对n_estimators调参

In [132]:
xgb_params = {'learning_rate': 0.1, 
              'n_estimators': 500, 
              'max_depth': 5, 
              'min_child_weight': 1, 
              'subsample': 0.8, 
              'colsample_bytree': 0.8, 
              'gamma': 0, 
              'reg_alpha': 0, 
              'reg_lambda': 1}
cv_params = {'n_estimators': [100, 300, 500, 700, 900]}

cv = StratifiedKFold(shuffle=True)
xgb_model = XGBClassifier(**xgb_params)
cv_xgb = GridSearchCV(xgb_model, cv_params, cv=cv, n_jobs=-1)
cv_xgb.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=5, min_child_weight=1,
                                     missing=None, n_estimators=500, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=0.8, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_estimators': [100, 300, 500, 700, 900]},
             pre_dispatch='2*n_job

In [133]:
cv_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=900, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [134]:
cv_xgb.best_score_

0.960724365792305

In [144]:
xgb_params = {'learning_rate': 0.1, 
              'n_estimators': 500, 
              'max_depth': 5, 
              'min_child_weight': 1, 
              'subsample': 0.8, 
              'colsample_bytree': 0.8, 
              'gamma': 0, 
              'reg_alpha': 0, 
              'reg_lambda': 1}
cv_params = {'n_estimators': [1300, 1400, 1500]}

cv = StratifiedKFold(shuffle=True)
xgb_model = XGBClassifier(**xgb_params)
cv_xgb = GridSearchCV(xgb_model, cv_params, cv=cv, n_jobs=-1)
cv_xgb.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=5, min_child_weight=1,
                                     missing=None, n_estimators=500, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=0.8, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_estimators': [1300, 1400, 1500]},
             pre_dispatch='2*n_jobs', ref

In [145]:
cv_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=1400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [146]:
cv_xgb.best_score_

0.962347006774926

##### 对min_child_weight以及max_depth调参

In [149]:
xgb_params = {'learning_rate': 0.1, 
              'n_estimators': 1400,  
              'subsample': 0.8, 
              'colsample_bytree': 0.8, 
              'gamma': 0, 
              'reg_alpha': 0, 
              'reg_lambda': 1}
cv_params = {'max_depth': [4, 6, 8], 
             'min_child_weight': [1, 3, 5]}

cv = StratifiedKFold(shuffle=True)
xgb_model = XGBClassifier(**xgb_params)
cv_xgb = GridSearchCV(xgb_model, cv_params, cv=cv, n_jobs=-1)
cv_xgb.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=1400, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=0.8, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': [4, 6, 8], 'min_child_weight': [1, 3, 5]},
             pre_disp

In [150]:
cv_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=1400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [151]:
cv_xgb.best_score_

0.9629502043291543

##### 对gamma调参

In [157]:
xgb_params = {'learning_rate': 0.1, 
              'n_estimators': 1400,
              'max_depth': 6,
              'min_child_weight': 1,
              'subsample': 0.8, 
              'colsample_bytree': 0.8, 
              'gamma': 0, 
              'reg_alpha': 0, 
              'reg_lambda': 1}
cv_params = {'gamma': [0.7, 0.8, 0.9]}

cv = StratifiedKFold(shuffle=True)
xgb_model = XGBClassifier(**xgb_params)
cv_xgb = GridSearchCV(xgb_model, cv_params, cv=cv, n_jobs=-1)
cv_xgb.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=6, min_child_weight=1,
                                     missing=None, n_estimators=1400, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=0.8, verbosity=1),
             iid='deprecated', n_jobs=-1, param_grid={'gamma': [0.7, 0.8, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_

In [158]:
cv_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.7,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=1400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [159]:
cv_xgb.best_score_

0.9625050496131321

##### 调参subsample以及colsample_bytree

In [160]:
xgb_params = {'learning_rate': 0.1, 
              'n_estimators': 1400,
              'max_depth': 6,
              'min_child_weight': 1,  
              'gamma': 0.7, 
              'reg_alpha': 0, 
              'reg_lambda': 1}
cv_params = {'subsample': [0.6, 0.7, 0.8, 0.9], 
              'colsample_bytree': [0.6, 0.7, 0.8, 0.9]}

cv = StratifiedKFold(shuffle=True)
xgb_model = XGBClassifier(**xgb_params)
cv_xgb = GridSearchCV(xgb_model, cv_params, cv=cv, n_jobs=-1)
cv_xgb.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0.7,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=6, min_child_weight=1,
                                     missing=None, n_estimators=1400, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
                         'subsample':

In [161]:
cv_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.7,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=1400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [162]:
cv_xgb.best_score_

0.963007653924684

##### 调参reg_alpha和reg_lambda

In [163]:
xgb_params = {'learning_rate': 0.1, 
              'n_estimators': 1400,
              'max_depth': 6,
              'subsample': 0.9,
              'colsample_bytree': 0.7,
              'min_child_weight': 1,  
              'gamma': 0.7, 
              'reg_alpha': 0, 
              'reg_lambda': 1}
cv_params = {'reg_alpha': [0.05, 0.1, 1, 2], 
             'reg_lambda': [0.05, 0.1, 1, 2]}

cv = StratifiedKFold(shuffle=True)
xgb_model = XGBClassifier(**xgb_params)
cv_xgb = GridSearchCV(xgb_model, cv_params, cv=cv, n_jobs=-1)
cv_xgb.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.7, gamma=0.7,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=6, min_child_weight=1,
                                     missing=None, n_estimators=1400, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=0.9, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'reg_alpha': [0.05, 0.1, 1, 2],
                         'reg_lambda': [0.0

In [164]:
cv_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.7,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=1400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.05, reg_lambda=0.1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [165]:
cv_xgb.best_score_

0.9631512186275966

##### 调参learning_rate

In [169]:
xgb_params = {'learning_rate': 0.1, 
              'n_estimators': 1400,
              'max_depth': 6,
              'subsample': 0.9,
              'colsample_bytree': 0.7,
              'min_child_weight': 1,  
              'gamma': 0.7, 
              'reg_alpha': 0.05, 
              'reg_lambda': 0.1}
cv_params = {'learning_rate': [0.005, 0.01, 0.05, 0.1]}

cv = StratifiedKFold(shuffle=True)
xgb_model = XGBClassifier(**xgb_params)
cv_xgb = GridSearchCV(xgb_model, cv_params, cv=cv, n_jobs=-1)
cv_xgb.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.7, gamma=0.7,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=6, min_child_weight=1,
                                     missing=None, n_estimators=1400, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0.05,
                                     reg_lambda=0.1, scale_pos_weight=1,
                                     seed=None, silent=None, subsample=0.9,
                                     verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.005, 0.01, 0.

In [170]:
cv_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.7,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=1400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.05, reg_lambda=0.1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [171]:
cv_xgb.best_score_

0.9626486452478249

In [172]:
xgb_params = {'learning_rate': 0.1, 
              'n_estimators': 1400,
              'max_depth': 6,
              'subsample': 0.9,
              'colsample_bytree': 0.7,
              'min_child_weight': 1,  
              'gamma': 0.7, 
              'reg_alpha': 0.05, 
              'reg_lambda': 0.1,
              'learning_rate': 0.1}
cv = StratifiedKFold(shuffle=True)
xgb_model = XGBClassifier(**xgb_params)
xgb_model.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.7,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=1400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.05, reg_lambda=0.1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [173]:
print(f'Accuracy with Xgboost: {xgb_model.score(test_x, test_y)}')
y_pred_xgb = xgb_model.predict(test_x)
print(classification_report(test_y, y_pred_xgb, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_xgb), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with Xgboost: 0.9626076967260195
              precision    recall  f1-score   support

        非新华社       0.90      0.69      0.78      1677
         新华社       0.97      0.99      0.98     15733

    accuracy                           0.96     17410
   macro avg       0.93      0.84      0.88     17410
weighted avg       0.96      0.96      0.96     17410



,非新华社,新华社
非新华社,1160,517
新华社,134,15599


In [180]:
xgb_model.fit(X_smote2, Y_smote2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.7,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=1400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.05, reg_lambda=0.1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [181]:
print(f'Accuracy with Xgboost: {xgb_model.score(test_x, test_y)}')
y_pred_xgb = xgb_model.predict(test_x)
print(classification_report(test_y, y_pred_xgb, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_xgb), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with Xgboost: 0.915393452039058
              precision    recall  f1-score   support

        非新华社       0.54      0.92      0.68      1677
         新华社       0.99      0.92      0.95     15733

    accuracy                           0.92     17410
   macro avg       0.76      0.92      0.81     17410
weighted avg       0.95      0.92      0.92     17410



,非新华社,新华社
非新华社,1537,140
新华社,1333,14400


### 总结

In [188]:
import cufflinks as cf
import matplotlib.pyplot as plt
%matplotlib inline
cf.set_config_file(offline=True)

model_name = ['Bayes', 'LR', 'SVM', 'RF', 'Xgboost']
result = {
    'Accuracy': [0.951, 0.909, 0.928, 0.959, 0.963],
    'Accuracy_with_smote': [0.867, 0.913, 0.927, 0.900, 0.915],
    'Recall': [0.77, 0.94, 0.85, 0.64, 0.69],
    'Recall_with_smote': [0.98, 0.92, 0.86, 0.92, 0.92]
        }

In [189]:
result = pd.DataFrame(result, index=model_name)
result

,Accuracy,Accuracy_with_smote,Recall,Recall_with_smote
Bayes,0.951,0.867,0.77,0.98
LR,0.909,0.913,0.94,0.92
SVM,0.928,0.927,0.85,0.86
RF,0.959,0.900,0.64,0.92
Xgboost,0.963,0.915,0.69,0.92


In [193]:
result.iplot(kind='scatter', mode='lines+markers+text', size=5, colors=['red', 'orange', 'blue', 'cyan'])

##### RandomForest与Xgboost的准确率最高；“非新华社”的召回率普遍偏低（可能因为样本不平衡），经过上采样后召回率能有提升